In [ ]:
# for training we need to lead the data and the model

# Train with objective: (These are the available tools: tool1, tool2, ...) (list of 3 to 7 tools?) [DATA WITH API CALLS] dont compute arguments + response tokens in loss

# Train argument generation mode: Tool explanation prompt with examples (?) [DATA UNTIL ARGUMENTS] only compute arguments in loss

# New tokens: API START, and API END

In [1]:
import torch
import torch.nn as nn

import beartype

@beartype
class ToolMaster(nn.Module):
    def __init__(self, 
                gptj_model: nn.Module,
                ):

        self.model = gptj_model

TypeError: 'module' object is not callable

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [2]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")



[2023-07-13 16:49:00,958] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/vol/bitbucket/jg2619/toolformer-luci/oldtoolvenv/lib/python3.10/site-packages/torch/cuda/__init__.py:152: UserWarning: 
    Found GPU0 NVIDIA GeForce GT 730 which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(old_gpu_warn % (d, name, major, minor, min_arch // 10, min_arch % 10))


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel


cache_dir = "/vol/bitbucket/jg2619/toolformer/cache"
cache_option = {"cache_dir": cache_dir} if cache_dir else {} 

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium', **cache_option)
model = GPT2LMHeadModel.from_pretrained('gpt2-medium', **cache_option)

tokenizer.add_tokens(['[PAD]', '<TOOL>', '</TOOL>'])
tokenizer.pad_token = '[PAD]'

model.resize_token_embeddings(len(tokenizer))

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

#gpt2tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
#gpt2model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

cache_dir = "/vol/bitbucket/jg2619/toolformer/cache"
cache_option = {"cache_dir": cache_dir} if cache_dir else {} 

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B", truncate=True, max_length=270, **cache_option)
tokenizer.add_tokens(['[PAD]', '<TOOL>', '</TOOL>'])
tokenizer.pad_token = '[PAD]'

model = model = AutoModelForCausalLM.from_pretrained(
            "EleutherAI/gpt-j-6B",
            revision="float16",
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True, **cache_option
        ).cuda()

model.resize_token_embeddings(len(tokenizer))

In [ ]:
from datasets import load_dataset
from transformers import Trainer
import os

dir_train = "/vol/bitbucket/jg2619/data/augmented_prompttrick/wikiSearch/"

# List files in dir_train
files = os.listdir(dir_train)

train_files = files[:int(len(files)*0.8)]
validation_files = files[int(len(files)*0.8):]

train_dataset = load_dataset("dir_train", data_files=train_files, split="train", cache_dir="/vol/bitbucket/jg2619/toolformer/cache")
validation_dataset = load_dataset("dir_train", data_files=validation_files, split="test", cache_dir="/vol/bitbucket/jg2619/toolformer/cache")

 
trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)



In [3]:
import os
from torch.utils.data import DataLoader
from datasets import load_dataset

cache_dir = "/vol/bitbucket/jg2619/toolformer/cache"

dataset_dir = "/vol/bitbucket/jg2619/data/preprocessed/big_load"


file_batch = 0
file_batch_size = 1   # number of files to load at a time
file_list = [file for file in os.listdir(dataset_dir) if file.endswith('.csv')]

print(file_list)


dataset = load_dataset(dataset_dir, cache_dir=cache_dir, data_files = file_list[:file_batch_size], split="train")
#dataset.set_format("torch")
dataloader = DataLoader(dataset, batch_size=1000)
data_iter = iter(dataloader)

Found cached dataset csv (/vol/bitbucket/jg2619/toolformer/cache/csv/big_load-bf33f16771d5e516/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


['calculator9.csv', 'calculator7.csv', 'calculator0.csv', 'calculator1.csv', 'calculator6.csv', 'calculator8.csv', 'calculator5.csv', 'calculator2.csv', 'calculator3.csv', 'calculator4.csv']


In [2]:
next(data_iter)

TypeError: must be real number, not NoneType

In [4]:
for data in dataloader:
    print(data)
    break

TypeError: must be real number, not NoneType

In [ ]:
import 